<h1>Welcome to the Resource Watch Duplicate Vector Layers Tool!</h1>

In [ ]:
#Import necessary layers
#!pip install LMIPy
import LMIPy as lmi
import os
from IPython.display import display
import numpy as np
import dotenv
import re

<font color=red>**ACTION REQUIRED** <br>
<font color=blue>**Enter the path to your .env file below.**</font>

In [ ]:
#insert the location of your .env file here:
dotenv.load_dotenv('')

#API token needed to make changes
API_TOKEN = os.getenv('RW_API_KEY')
if API_TOKEN:
    print('Your .env successfully loaded!')
else:
    print('Please check the path to your .env file and make sure you have a key called RW_API_KEY in your .env file.')

<font color=red>**ACTION REQUIRED** <br>
<font color=blue>**Enter your dataset ID below** <br>
Run the next cell to show the dataset that you have selected, its first layer, and the total number of layers in the dataset.</font>

In [ ]:
#Select dataset to edit
dataset_id = '' #NEEDS TO BE THE LONG ONE LIKE 6cfd6255-609e-4922-8709-5aaad0db9fae
dataset = lmi.Dataset(dataset_id)
long_dataset_id = dataset.id

print('Dataset selected: ')
display(dataset)

layers = dataset.layers
example_layer = layers[0]   #Chose the first layer

print('First layer:')
display(example_layer)

print("Number of layers in dataset: {}".format(len(dataset.layers)))

<font color=blue>Run the next cell to view the attributes that you may want to change.<br>
We will print the attributes for the first layer to use as an example.</font>

In [ ]:
#Gather the attributes that might need to change
ex_name = example_layer.attributes['name']
ex_description = example_layer.attributes['description']
ex_appConfig = example_layer.attributes['layerConfig']
ex_account = ex_appConfig['account']
ex_sql = ex_appConfig['body']['layers'][0]['options']['sql']
ex_cartocss = ex_appConfig['body']['layers'][0]['options']['cartocss']
ex_vectorLayers = ex_appConfig['body']['vectorLayers']
ex_legendConfig = example_layer.attributes['legendConfig']
ex_interactionConfig = example_layer.attributes['interactionConfig']

#Print them out
print('name: \n{} \n'.format(ex_name))
print('description: \n{} \n'.format(ex_description))
print('account: \n{} \n'.format(ex_account))
print('sql: \n{} \n'.format(ex_sql))
print('cartocss: \n{} \n'.format(ex_cartocss))
print('vectorLayers: \n{} \n'.format(ex_vectorLayers))
print('legendConfig: \n{} \n'.format(ex_legendConfig))
print('interactionConfig: \n{} \n'.format(ex_interactionConfig))

<h1>Define our replacement.</h1>

<font color=red>**ACTION REQUIRED**</font> <br>
<font color=blue>**Enter the field you would like to edit (from those printed above), the new entry for that field, and indicate whether or not there is a variable year component to the layers**

In [ ]:
# Choose the field you want to edit
field_to_edit = '' #e.g. 'name', 'description', 'sql', 'interactionConfig', etc.

# Copy the current entry for this field from above.
# If you are editing the legendConfig, interactionConfig, or vectorLayers, this should be a dictionary.
# For any other field, paste the entry inside quotation marks to make it a string.
# Edit this field however you would like to see it updated.
ex_replacement_text = \
''

# If there is a year in this field that changes from layer to layer, please type the year that was used in your 
# example replacement text below (as a string).
# If there is no varying year component in this field, just leave the field below empty ('')
ex_variable_year = '' #e.g., '1990' or ''

<font color=red>**ACTION REQUIRED**</font> <br>
<font color=blue>**Run this cell and make sure that the replacement text (printed below) looks ok for the example layer.**

In [ ]:
# get the original text in the field we are changing
field_to_edit_text = globals()['ex_'+field_to_edit]
# If there is a variable string to maintain
if len(ex_variable_year):
    # generate a list of words in this string
    word_list = re.findall(r"[\w']+|[.,!?;]", field_to_edit_text)
    # find the index of the year in this list of words
    year_idx = word_list.index(ex_variable_year)

    # generate the variable year to keep for the current layer, based on the location of the text in the example
    variable_year = word_list[year_idx]
    # replace the year in the example replacement text with a placeholder
    replacement_text_variable = ex_replacement_text.replace(variable_year, '{string_to_keep}')
    # fill in the placeholder with the variable string for the current layer
    replacement_text = replacement_text_variable.format(string_to_keep=variable_year)
# If there is NOT a variable string to maintain
else:
    # just use the replacement text that was entered above exactly as it is
    replacement_text=ex_replacement_text

#Print out the values so you can double check them
print(f'Original text: \n{field_to_edit_text}')
print(f'\nVariable year: {ex_variable_year}')
print(f'\nReplacement text:\n{replacement_text}')

<h1>Loop through all layers and edit the selected field.</h1>

<font color=blue>**The cell below loops through each of the layers in the dataset and replaces the selected field with your replacement text. If you indicated that there was a year to keep in each layer, that year is identified and inserted into your replacement text before it is updated.**</font>

In [ ]:
for layer in layers:
    print('Updating layer:')
    display(layer) 
    
    #Gather the attributes that might need to change
    name = layer.attributes['name']
    description = layer.attributes['description']
    appConfig = layer.attributes['layerConfig']
    account = appConfig['account']
    sql = appConfig['body']['layers'][0]['options']['sql']
    cartocss = appConfig['body']['layers'][0]['options']['cartocss']
    vectorLayers = appConfig['body']['vectorLayers']
    legendConfig = layer.attributes['legendConfig']
    interactionConfig = layer.attributes['interactionConfig']
    
    if len(ex_variable_year):
        # get the original text in the field we are changing
        field_to_edit_text = globals()[field_to_edit]
        
        # get the list of words in the field for this layer
        word_list = re.findall(r"[\w']+|[.,!?;]", field_to_edit_text)

        # generate the variable year to keep for the current layer, based on the location of the text in the example
        variable_year = word_list[year_idx]        
        # fill in the placeholder with the variable string for the current layer
        replacement_text = replacement_text_variable.format(string_to_keep=variable_year)
    else:
        # just use the replacement text that was entered above exactly as it is
        replacement_text=ex_replacement_text

    # Replace layer attributes with new values
    if field_to_edit=='sql' or field_to_edit=='cartocss':
        appConfig = layer.attributes['layerConfig']
        appConfig['body']['layers'][0]['options'][field_to_edit] = replacement_text
        payload = {
        'layerConfig': {
            **appConfig
            }
        }
    elif field_to_edit=='account':
        appConfig = layer.attributes['layerConfig']
        appConfig[field_to_edit] = replacement_text
        payload = {
        'layerConfig': {
            **appConfig
            }
        }
    elif field_to_edit=='vectorLayers':
        appConfig = layer.attributes['layerConfig']
        appConfig['body']['layers'][0]['options'][field_to_edit] = replacement_text
        payload = {
        'layerConfig': {
            **appConfig
            }
        }
    else:        
        payload = {
            field_to_edit: replacement_text
        }
    # Update the layer
    layer = layer.update(update_params=payload, token=API_TOKEN)
    print(f'\nUpdated: {layer}')
    print('\n')